In [1]:
using DrWatson
@quickactivate "MEngProject"
using MEngProject,
    CUDA,
    DifferentialEquations,
    PyPlot,
    NNlib,
    ImageFiltering,
    Images,
    ImageIO,
    MEngProject.LaminartKernels,
    MEngProject.LaminartInitFunc,
    MEngProject.Utils,
    BenchmarkTools,
    Test

using OrdinaryDiffEq,
    ParameterizedFunctions, LSODA, Sundials, DiffEqDevTools, Noise

# Solver


In [34]:
batch = 9901



    
global benchm_s = []
global alg_ = []
global stats_ = []
global gpu_ = []


tspan = (0.0f0, 1f0)

batch_ = string(batch, "_", rand(1000:9999))
mkdir(plotsdir(string("bench_solver", batch_)))
file = "kan_sq_cont_l.png"

solvers = [
            AutoTsit5(Rosenbrock23()),
            Tsit5(),
            BS3(),
            lsoda(),
            Vern7(),
            ]
# alg=lsoda()

5-element Array{DiffEqBase.AbstractODEAlgorithm,1}:
 CompositeAlgorithm{Tuple{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType}},AutoSwitch{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType},Rational{Int64},Int64}}((Tsit5(), Rosenbrock23{0,true,DefaultLinSolve,DataType}(DefaultLinSolve(nothing, nothing), Val{:forward})), AutoSwitch{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType},Rational{Int64},Int64}(0, Tsit5(), Rosenbrock23{0,true,DefaultLinSolve,DataType}(DefaultLinSolve(nothing, nothing), Val{:forward}), false, 10, 3, 9//10, 9//10, 2, false))
 Tsit5()
 BS3()
 lsoda()
 Vern7(true)

In [35]:


file = "kan_sq_cont_l.png"

solvers = [
            AutoTsit5(Rosenbrock23()),
            Tsit5(),
            BS3(),
            lsoda(),
            Vern7(),
            ]
# alg=lsoda()

# GPU
p = LaminartInitFunc.parameterInit_conv_gpu(
    datadir("img", file),
    Parameters.parameters_f32,
);

u0 = cu(reshape(
    zeros(Float32, p.dim_i, p.dim_j * (5 * p.K + 2)),
    p.dim_i,
    p.dim_j,
    5 * p.K + 2,
    1,
))

arr1 = similar(@view u0[:, :, 1:2, :])
arr2 = similar(@view u0[:, :, 1:1, :])
f = LaminartFunc.LamFunction(
    arr1, #x
    similar(arr1), #m
    similar(arr1), #s
    arr2, #x_lgn,
    similar(arr1), #C,
    similar(arr1), #H_z,
    similar(arr1), # dy_temp,
    similar(arr1), # dm_temp,
    similar(arr1), # dz_temp,
    similar(arr1), # ds_temp,
    similar(arr2), # dv_temp,
    similar(arr1), # H_z_temp,
    similar(arr2), #  V_temp_1,
    similar(arr2), #  V_temp_2,
    similar(arr1), #  A_temp,
    similar(arr1), #   B_temp
)
global prob_s = ODEProblem(f, u0, tspan, p)

ODEProblem with uType CuArray{Float32,4} and tType Float32. In-place: true
timespan: (0.0f0, 1.0f0)
u0: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

# AutoTsit5(Rosenbrock23()),


In [36]:
# algg = solvers[1]
# # for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_s, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_s, algg_))
#         push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# # end

# Tsit5(),
            

In [37]:
algg = solvers[2]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "GPU")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  87
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          14
Number of rejected steps:                          0

1-element Array{Any,1}:
 "GPU"

#         BS3(),

In [38]:
algg = solvers[3]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "GPU")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  57
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          18
Number of rejected steps:                          0

2-element Array{Any,1}:
 "GPU"
 "GPU"

#    lsoda(),
         

In [39]:
# algg = solvers[4]
# # for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_s, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_s, algg_))
#         push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# # end

#     Vern7(),

In [40]:
algg = solvers[5]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "GPU")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  192
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          19
Number of rejected steps:                          0

3-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"

# CPU conv

In [41]:
# # tspan = (0.0f0, 800f0)


# file = "kan_sq_cont_l.png"

# solvers = [
#             AutoTsit5(Rosenbrock23()),
#             Tsit5(),
#             BS3(),
#             lsoda(),
#             Vern7(),
#             ]
# # alg=lsoda()

# CPU
p = LaminartInitFunc.parameterInit_conv_cpu(
    datadir("img", file),
    Parameters.parameters_f32,
);

u0 = reshape(
    zeros(Float32, p.dim_i, p.dim_j * (5 * p.K + 2)),
    p.dim_i,
    p.dim_j,
    5 * p.K + 2,
    1,
)

arr1 = similar(@view u0[:, :, 1:2, :])
arr2 = similar(@view u0[:, :, 1:1, :])
f = LaminartFunc.LamFunction(
    arr1, #x
    similar(arr1), #m
    similar(arr1), #s
    arr2, #x_lgn,
    similar(arr1), #C,
    similar(arr1), #H_z,
    similar(arr1), # dy_temp,
    similar(arr1), # dm_temp,
    similar(arr1), # dz_temp,
    similar(arr1), # ds_temp,
    similar(arr2), # dv_temp,
    similar(arr1), # H_z_temp,
    similar(arr2), #  V_temp_1,
    similar(arr2), #  V_temp_2,
    similar(arr1), #  A_temp,
    similar(arr1), #   B_temp
)
global prob_s = ODEProblem(f, u0, tspan, p)

ODEProblem with uType Array{Float32,4} and tType Float32. In-place: true
timespan: (0.0f0, 1.0f0)
u0: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

# AutoTsit5(Rosenbrock23()),


In [42]:
# algg = solvers[1]
# # for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_s, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_s, algg_))
#         push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# # end

# Tsit5(),
            

In [43]:
algg = solvers[2]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "CPU conv")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  87
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          14
Number of rejected steps:                          0

4-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"

#         BS3(),

In [44]:
algg = solvers[3]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "CPU Conv")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  54
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          17
Number of rejected steps:                          0

5-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"

#    lsoda(),
         

In [45]:
# algg = solvers[4]
# # for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_s, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_s, algg_))
#         push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# # end

#     Vern7(),

In [46]:
algg = solvers[5]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "CPU conv")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  192
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          19
Number of rejected steps:                          0

6-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"
 "CPU conv"

# CPU imfil

In [47]:
# CPU imfil 

p = LaminartInitFunc.parameterInit_imfil_cpu(
    datadir("img", file),
    Parameters.parameters_f32,
);

u0 = reshape(zeros(Float32, p.dim_i, p.dim_j*(5*p.K+2)), p.dim_i, p.dim_j, 5*p.K+2);


arr1 = u0[:, :, 1:p.K]
arr2 = u0[:, :, 1:1];


f = LaminartFunc.LamFunction_imfil_cpu(
 similar(arr2[:,:,1]), #x_lgn,
arr1, #C,
similar(arr1), #H_z,
    similar(arr1), # H_z_temp,
    similar(arr2[:,:,1]), # v_C_temp1,
    similar(arr2[:,:,1]), # v_C_temp2,
    similar(arr1), # v_C_tempA,
    similar(arr1[:,:,1]), #W_temp
    );


global prob_sc = ODEProblem(f, u0, tspan, p)

ODEProblem with uType Array{Float32,3} and tType Float32. In-place: true
timespan: (0.0f0, 1.0f0)
u0: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [48]:
# 

# AutoTsit5(Rosenbrock23()),


In [49]:
algg = solvers[2]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_sc, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_sc, algg_))
        push!(gpu_, "CPU imfil")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  87
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          14
Number of rejected steps:                          0

7-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"
 "CPU conv"
 "CPU imfil"

# Tsit5 


In [50]:
algg = solvers[2]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_sc, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_sc, algg_))
        push!(gpu_, "CPU imfil")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  87
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          14
Number of rejected steps:                          0

8-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"
 "CPU conv"
 "CPU imfil"
 "CPU imfil"

# BS3 


In [51]:
algg = solvers[3]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_sc, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_sc, algg_))
        push!(gpu_, "CPU imfil")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  54
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          17
Number of rejected steps:                          0

9-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"
 "CPU conv"
 "CPU imfil"
 "CPU imfil"
 "CPU imfil"

# lsoda 


In [52]:
# algg = solvers[4]
# # for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_sc, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_sc, algg_))
#         push!(gpu_, "CPU imfil")
#     catch err
#         print(err)
#     end
# # end

# vern7 


In [53]:
algg = solvers[5]
# for algg in solvers
    @. u0 = 0.0f0
    try
        global algg_ = algg
        sol = solve(prob_sc, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_sc, algg_))
        push!(gpu_, "CPU imfil")
    catch err
        print(err)
    end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  192
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          19
Number of rejected steps:                          0

10-element Array{Any,1}:
 "GPU"
 "GPU"
 "GPU"
 "CPU conv"
 "CPU Conv"
 "CPU conv"
 "CPU imfil"
 "CPU imfil"
 "CPU imfil"
 "CPU imfil"

In [54]:
# # benchmark plot

# # time
# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         median(benchm_s[ben[1]].times) * 1e-9,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Time (\$s\$)")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_time.png"),
# ))
# close("all")




# # memory

# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         benchm_s[ben[1]].memory * 1e-6,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Memory (\$MB\$)")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_mem.png"),
# ))
# close("all")


# # alloc

# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         benchm_s[ben[1]].allocs,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Allocations")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_alloc.png"),
# ))
# close("all")


# end

In [70]:
# benchmark plot

# time
fig, ax = plt.subplots()
for ben in enumerate(gpu_)
   aa= ax.scatter(
        string(alg_[ben[1]]),
        median(benchm_s[ben[1]].times) * 1e-9,
        label = string(ben[2]),
        color = Utils.colours[1],
        edgecolors = "none",
    )
end
for ben in enumerate(gpu_[4:6])
   bb=ax.scatter(
        string(alg_[ben[1]]),
        median(benchm_s[ben[1]].times) * 1e-9,
#         label = string(ben[2]),
        color = Utils.colours[2],
        edgecolors = "none",
    )
end
for ben in enumerate(gpu_[7:9])
  cc= ax.scatter(
        string(alg_[ben[1]]),
        median(benchm_s[ben[1]].times) * 1e-9,
#         label = string(ben[2]),
        color = Utils.colours[3],
        edgecolors = "none",
    )
end
plt.legend(("GPU", "CPU conv", "CPU imfilter"))
ax.set_ylabel("Time (\$s\$)")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_time.png"),
))
close("all")




# memory

fig, ax = plt.subplots()
for ben in enumerate(gpu_)
    ax.scatter(
        string(alg_[ben[1]]),
        benchm_s[ben[1]].memory * 1e-6,
                label = ben[2],
        color = Utils.colours[ben[1]],
        edgecolors = "none",
    )
end
plt.legend()
ax.set_ylabel("Memory (\$MB\$)")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_mem.png"),
))
close("all")


# alloc

fig, ax = plt.subplots()
for ben in enumerate(gpu_)
    ax.scatter(
        string(alg_[ben[1]]),
        benchm_s[ben[1]].allocs,
                label = ben[2],
        color = Utils.colours[ben[1]],
        edgecolors = "none",
    )
end
plt.legend()
ax.set_ylabel("Allocations")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_alloc.png"),
))
close("all")


# end

In [80]:
# benchmark plot

# time
fig, ax = plt.subplots()

    ax.scatter(
        string(alg_[1]),
        median(benchm_s[1].times) * 1e-9,
        label = "GPU conv",
#         color = Utils.colours[1],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[2]),
        median(benchm_s[2].times) * 1e-9,
#         label = string(ben[2]),
        label = "GPU conv",
#         color = Utils.colours[1],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[3]),
        median(benchm_s[3].times) * 1e-9,
#         label = string(ben[2]),
        label = "GPU conv",
#         color = Utils.colours[1],
        edgecolors = "none",
    )
    ax.scatter(
        string(alg_[4]),
        median(benchm_s[4].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU conv",
#         color = Utils.colours[2],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[5]),
        median(benchm_s[5].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU conv",
#         color = Utils.colours[2],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[6]),
        median(benchm_s[6].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU conv",
#         color = Utils.colours[2],
        edgecolors = "none",
    )
    ax.scatter(
        string(alg_[7]),
        median(benchm_s[7].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU imfilter",
#         color = Utils.colours[3],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[8]),
        median(benchm_s[8].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU imfilter",
#         color = Utils.colours[3],
        edgecolors = "none",
    )
     ax.scatter(
        string(alg_[9]),
        median(benchm_s[9].times) * 1e-9,
#         label = string(ben[2]),
        label = "CPU imfilter",
#         color = Utils.colours[3],
        edgecolors = "none",
    )
plt.legend()
ax.set_ylabel("Time (\$s\$)")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_time.png"),
))
close("all")




# memory

fig, ax = plt.subplots()
for ben in enumerate(gpu_)
    ax.scatter(
        string(alg_[ben[1]]),
        benchm_s[ben[1]].memory * 1e-6,
                label = ben[2],
        color = Utils.colours[ben[1]],
        edgecolors = "none",
    )
end
plt.legend()
ax.set_ylabel("Memory (\$MB\$)")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_mem.png"),
))
close("all")


# alloc

fig, ax = plt.subplots()
for ben in enumerate(gpu_)
    ax.scatter(
        string(alg_[ben[1]]),
        benchm_s[ben[1]].allocs,
                label = ben[2],
        color = Utils.colours[ben[1]],
        edgecolors = "none",
    )
end
plt.legend()
ax.set_ylabel("Allocations")
ax.set_ylim(ymin = 0)
ax.grid(true)
fig.tight_layout()
plt.savefig(plotsdir(
    string("bench_solver", batch_),
    string("bench_solver_alloc.png"),
))
close("all")


# end

In [81]:
benchm_s[1]

BenchmarkTools.Trial: 
  memory estimate:  19.56 MiB
  allocs estimate:  405174
  --------------
  minimum time:     185.406 ms (0.00% GC)
  median time:      188.963 ms (0.00% GC)
  mean time:        204.786 ms (4.84% GC)
  maximum time:     327.762 ms (29.72% GC)
  --------------
  samples:          25
  evals/sample:     1

In [83]:
benchm_s[2]

BenchmarkTools.Trial: 
  memory estimate:  12.65 MiB
  allocs estimate:  264713
  --------------
  minimum time:     123.473 ms (0.00% GC)
  median time:      124.175 ms (0.00% GC)
  mean time:        131.947 ms (3.08% GC)
  maximum time:     228.434 ms (25.87% GC)
  --------------
  samples:          38
  evals/sample:     1

In [82]:
benchm_s[3]

BenchmarkTools.Trial: 
  memory estimate:  43.55 MiB
  allocs estimate:  896975
  --------------
  minimum time:     403.622 ms (0.00% GC)
  median time:      404.990 ms (0.00% GC)
  mean time:        424.405 ms (2.47% GC)
  maximum time:     485.025 ms (8.66% GC)
  --------------
  samples:          12
  evals/sample:     1

In [24]:
# # """
# # # script bench_solver

# # - Julia version: 1.4
# # - Author: niallcullinane
# # - Date: 2020-08-20


# # Script to benchmark GPU and CPU implementions of model.
# # # Examples

# # ```jldoctest
# # julia>
# # ```
# # """

# # using DrWatson
# # @quickactivate "MEngProject"
# # using MEngProject,
# #     CUDA,
# #     DifferentialEquations,
# #     PyPlot,
# #     NNlib,
# #     ImageFiltering,
# #     Images,
# #     ImageIO,
# #     MEngProject.LaminartKernels,
# #     MEngProject.LaminartInitFunc,
# #     MEngProject.Utils,
# #     BenchmarkTools,
# #     Test

# # using OrdinaryDiffEq,
# #     ParameterizedFunctions, LSODA, Sundials, DiffEqDevTools, Noise

# batch = 9000


# # global benchm_s = []
# # global benchm_sname = []
# # global alg_ = []
# # global stats_ = []
# # global gpu_ = []
# # let
    
# global benchm_s = []
# global alg_ = []
# global stats_ = []
# global gpu_ = []


# tspan = (0.0f0, 800f0)

# batch_ = string(batch, "_", rand(1000:9999))
# mkdir(plotsdir(string("bench_solver", batch_)))
# file = "kan_sq_cont_l.png"

# solvers = [
#             AutoTsit5(Rosenbrock23()),
#             Tsit5(),
#             BS3(),
#             lsoda(),
#             Vern7(),
#             ]
# # alg=lsoda()

# # GPU
# p = LaminartInitFunc.parameterInit_conv_gpu(
#     datadir("img", file),
#     Parameters.parameters_f32,
# );

# u0 = cu(reshape(
#     zeros(Float32, p.dim_i, p.dim_j * (5 * p.K + 2)),
#     p.dim_i,
#     p.dim_j,
#     5 * p.K + 2,
#     1,
# ))

# arr1 = similar(@view u0[:, :, 1:2, :])
# arr2 = similar(@view u0[:, :, 1:1, :])
# f = LaminartFunc.LamFunction(
#     arr1, #x
#     similar(arr1), #m
#     similar(arr1), #s
#     arr2, #x_lgn,
#     similar(arr1), #C,
#     similar(arr1), #H_z,
#     similar(arr1), # dy_temp,
#     similar(arr1), # dm_temp,
#     similar(arr1), # dz_temp,
#     similar(arr1), # ds_temp,
#     similar(arr2), # dv_temp,
#     similar(arr1), # H_z_temp,
#     similar(arr2), #  V_temp_1,
#     similar(arr2), #  V_temp_2,
#     similar(arr1), #  A_temp,
#     similar(arr1), #   B_temp
# )
# global prob_s = ODEProblem(f, u0, tspan, p)

# # alg = solvers[1]
# for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_s, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_s, algg_))
#         push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# end






# # CPU conv

# p = LaminartInitFunc.parameterInit_imfil_cpu(
#     datadir("img", file),
#     Parameters.parameters_f32,
# );

# u0 = reshape(zeros(Float32, p.dim_i, p.dim_j*(5*p.K+2)), p.dim_i, p.dim_j, 5*p.K+2);


# arr1 = u0[:, :, 1:p.K]
# arr2 = u0[:, :, 1:1];


# f = LaminartFunc.LamFunction_imfil_cpu(
#  similar(arr2[:,:,1]), #x_lgn,
# arr1, #C,
# similar(arr1), #H_z,
#     similar(arr1), # H_z_temp,
#     similar(arr2[:,:,1]), # v_C_temp1,
#     similar(arr2[:,:,1]), # v_C_temp2,
#     similar(arr1), # v_C_tempA,
#     similar(arr1[:,:,1]), #W_temp
#     );


# global prob_sc = ODEProblem(f, u0, tspan, p)
# for algg in solvers
#     @. u0 = 0.0f0
#     try
#         global algg_ = algg
#         sol = solve(prob_sc, algg)
#         push!(alg_, sol.alg)
#         push!(stats_, sol.destats)
#         print(sol.destats)
#         push!(benchm_s, @benchmark solve(prob_sc, algg_))
#         push!(gpu_, "CPU")
#     catch err
#         print(err)
#     end
# end



# # benchmark plot

# # time
# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         median(benchm_s[ben[1]].times) * 1e-9,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Time (\$s\$)")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_time.png"),
# ))
# close("all")




# # memory

# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         benchm_s[ben[1]].memory * 1e-6,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Memory (\$MB\$)")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_mem.png"),
# ))
# close("all")


# # alloc

# fig, ax = plt.subplots()
# for ben in enumerate(gpu_)
#     ax.scatter(
#         string(ben[2], " ", alg_[ben[1]]),
#         benchm_s[ben[1]].allocs,
#         color = Utils.colours[ben[1]],
#         edgecolors = "none",
#     )
# end

# ax.set_ylabel("Allocations")
# ax.set_ylim(ymin = 0)
# ax.grid(true)
# fig.tight_layout()
# plt.savefig(plotsdir(
#     string("bench_solver", batch_),
#     string("bench_solver_alloc.png"),
# ))
# close("all")


# # end




# Tsit vs BS3

In [25]:
batch = 9900



    
global benchm_s = []
global alg_ = []
global stats_ = []
global gpu_ = []


tspan = (0.0f0, 800f0)

batch_ = string(batch, "_", rand(1000:9999))
mkdir(plotsdir(string("solvercomp", batch_)))
file = "kan_sq_cont_l.png"

solvers = [
            AutoTsit5(Rosenbrock23()),
            Tsit5(),
            BS3(),
            lsoda(),
            Vern7(),
            ]
# alg=lsoda()

5-element Array{DiffEqBase.AbstractODEAlgorithm,1}:
 CompositeAlgorithm{Tuple{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType}},AutoSwitch{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType},Rational{Int64},Int64}}((Tsit5(), Rosenbrock23{0,true,DefaultLinSolve,DataType}(DefaultLinSolve(nothing, nothing), Val{:forward})), AutoSwitch{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType},Rational{Int64},Int64}(0, Tsit5(), Rosenbrock23{0,true,DefaultLinSolve,DataType}(DefaultLinSolve(nothing, nothing), Val{:forward}), false, 10, 3, 9//10, 9//10, 2, false))
 Tsit5()
 BS3()
 lsoda()
 Vern7(true)

In [26]:


file = "kan_sq_cont_l.png"

solvers = [
            AutoTsit5(Rosenbrock23()),
            Tsit5(),
            BS3(),
            lsoda(),
            Vern7(),
            ]
# alg=lsoda()

# GPU
p = LaminartInitFunc.parameterInit_conv_gpu(
    datadir("img", file),
    Parameters.parameters_f32,
);

u0 = cu(reshape(
    zeros(Float32, p.dim_i, p.dim_j * (5 * p.K + 2)),
    p.dim_i,
    p.dim_j,
    5 * p.K + 2,
    1,
))

arr1 = similar(@view u0[:, :, 1:2, :])
arr2 = similar(@view u0[:, :, 1:1, :])
f = LaminartFunc.LamFunction(
    arr1, #x
    similar(arr1), #m
    similar(arr1), #s
    arr2, #x_lgn,
    similar(arr1), #C,
    similar(arr1), #H_z,
    similar(arr1), # dy_temp,
    similar(arr1), # dm_temp,
    similar(arr1), # dz_temp,
    similar(arr1), # ds_temp,
    similar(arr2), # dv_temp,
    similar(arr1), # H_z_temp,
    similar(arr2), #  V_temp_1,
    similar(arr2), #  V_temp_2,
    similar(arr1), #  A_temp,
    similar(arr1), #   B_temp
)
global prob_s = ODEProblem(f, u0, tspan, p)

ODEProblem with uType CuArray{Float32,4} and tType Float32. In-place: true
timespan: (0.0f0, 800.0f0)
u0: Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

# Tsit5(),
            

In [27]:
algg = solvers[2]
# for algg in solvers
    @. u0 = 0.0f0
#     try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  7851
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          1303
Number of rejected steps:                          5

1-element Array{Any,1}:
 "GPU"

In [28]:
        # plots
        for t ∈ [25, 50, 100, 200, 400, 800]
            Utils.plot_k2(sol, t, "solvercomp", batch_, "tsit")
        end


        Utils.plot_t_act(sol, "solvercomp", batch_, "tsit")


┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /mnt/storage_1/users/cullinanen/.julia/packages/GPUArrays/eVYIC/src/host/indexing.jl:43


#         BS3(),

In [29]:
algg = solvers[3]
# for algg in solvers
    @. u0 = 0.0f0
#     try
        global algg_ = algg
        sol = solve(prob_s, algg)
        push!(alg_, sol.alg)
        push!(stats_, sol.destats)
        print(sol.destats)
        push!(benchm_s, @benchmark solve(prob_s, algg_))
        push!(gpu_, "GPU")
#     catch err
#         print(err)
#     end
# end

DiffEqBase.DEStats
Number of function 1 evaluations:                  4782
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          1589
Number of rejected steps:                          4

2-element Array{Any,1}:
 "GPU"
 "GPU"

In [30]:
        # plots
        for t ∈ [25, 50, 100, 200, 400, 800]
            Utils.plot_k2(sol, t, "solvercomp", batch_, "bs3")
        end


        Utils.plot_t_act(sol, "solvercomp", batch_, "bs3")

In [33]:
 alg_[3]

BoundsError: BoundsError: attempt to access 2-element Array{Any,1} at index [3]